In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Load the TFLite model
model_path = "manipulation_detector_config_2.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocesses an image before it is given to the model to ensure consistency
# across all images that the model is learning off of.
def preprocess_image(image_path):
    # Load the image and convert it to RGB colour space; 
    # we want all images to be in same colour space for consistency.
    with Image.open(image_path) as image:
        image = image.convert("RGB")
    image = image.resize((299, 299))
    # Convert the image to an array. Normalize its pixel values to be between 0 and 1.
    # Since most images in the training/test dataset are jpegs and in RGB, assume that
    # we can divide each pixel by 255 to normalize them to be between 0 and 1.
    preprocessed_image = np.array(image) / 255.0
    # Return the preprocessed image.
    return preprocessed_image

# Define a function to classify a single image
def classify_image(image_path):
    input_data = preprocess_image(image_path)
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data[0][0]

# Define the folder where the images are located
folder_path = "reserve"

# Initialize counters for classification results
num_authentic = 0
num_manipulated = 0
num_au_authentic = 0
num_au_manipulated = 0
num_tp_authentic = 0
num_tp_manipulated = 0
num_au_filenames = 0
num_tp_filenames = 0

# Iterate over all the images in the folder
for filename in os.listdir(folder_path):
    if (filename.endswith(".jpg") or filename.endswith(".jpeg")):
        # Classify the image and update counters
        image_path = os.path.join(folder_path, filename)
        classification_result = classify_image(image_path)
        if classification_result < 0.5:
            print(f"{filename} - Manipulated: {classification_result}")
            num_manipulated += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_manipulated += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_manipulated += 1
                num_tp_filenames += 1
        else:
            print(f"{filename} - Authentic: {classification_result}")
            num_authentic += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_authentic += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_authentic += 1
                num_tp_filenames += 1

# Print the classification results and filename counts
print(f"Total authentic images: {num_authentic}")
print(f"Total manipulated images: {num_manipulated}")
print(f"Number of 'au' authentic images: {num_au_authentic}")
print(f"Number of 'au' manipulated images: {num_au_manipulated}")
print(f"Number of 'tp' authentic images: {num_tp_authentic}")
print(f"Number of 'tp' manipulated images: {num_tp_manipulated}")
print(f"Number of 'au' filenames: {num_au_filenames}")
print(f"Number of 'tp' filenames: {num_tp_filenames}")

precision = num_au_authentic/(num_au_authentic + num_tp_authentic)
recall = num_au_authentic/(num_au_authentic + num_au_manipulated)
accuracy = (num_au_authentic + num_tp_manipulated)/(num_au_authentic + num_tp_authentic + num_au_manipulated + num_tp_manipulated)

print(f"precision: {precision}")
print(f"recall: {recall}")
print(f"accuracy: {accuracy}")



Au_pla_30535.jpg - Authentic: 1.0
Tp_S_NRD_S_N_pla20018_pla20018_02489.jpeg - Manipulated: 0.09877839684486389
Au_pla_30253.jpg - Authentic: 0.9999995231628418
Au_pla_20040.jpg - Manipulated: 0.0007033618749119341
Au_pla_30247.jpg - Authentic: 0.9999988675117493
Au_pla_20054.jpg - Manipulated: 0.008737806230783463
Au_pla_30521.jpg - Authentic: 1.0
Tp_S_NRN_S_N_pla00065_pla00065_01172.jpeg - Authentic: 1.0
Au_pla_30509.jpg - Authentic: 0.9999983310699463
Au_arc_30592.jpg - Authentic: 1.0
Tp_S_NRN_S_N_ind00055_ind00055_00905.jpeg - Authentic: 1.0
Tp_D_CRN_S_N_sec00071_art00028_11281.jpg - Authentic: 0.7881886959075928
Tp_D_NNN_L_N_ani00010_nat00095_10218.jpeg - Authentic: 0.994405210018158
Au_pla_20068.jpg - Manipulated: 0.116404689848423
Au_arc_30586.jpg - Authentic: 1.0
Au_nat_30570.jpg - Authentic: 1.0
Au_nat_30216.jpg - Authentic: 0.9999999403953552
Au_nat_20005.jpg - Manipulated: 0.1982526332139969
Au_pla_30290.jpg - Authentic: 1.0
Au_pla_20083.jpg - Manipulated: 0.05364349856972694

Au_sec_30432.jpg - Authentic: 1.0
Tp_S_NRN_S_N_pla20030_pla20030_01930.jpeg - Manipulated: 0.36629369854927063
Au_nat_30613.jpg - Authentic: 1.0
Au_pla_30695.jpg - Authentic: 1.0
Tp_S_NRD_S_N_pla20007_pla20007_01907.jpeg - Manipulated: 0.053445376455783844
Au_pla_00030.jpg - Authentic: 0.9999999403953552
Tp_D_CRN_S_N_cha10142_cha00086_12235.jpg - Manipulated: 0.06733059883117676
Au_nat_30175.jpg - Authentic: 1.0
Au_nat_30161.jpg - Authentic: 1.0
Au_sec_30368.jpg - Authentic: 1.0
Au_pla_00024.jpg - Authentic: 1.0
Au_pla_30681.jpg - Authentic: 1.0
Au_nat_30607.jpg - Authentic: 0.9999814033508301
Au_txt_00017.jpg - Manipulated: 0.22851614654064178
Tp_D_NNN_M_B_nat10128_ani00058_11541.jpg - Manipulated: 0.15374793112277985
Au_sec_30383.jpg - Authentic: 1.0
Tp_S_NRN_S_N_arc20057_arc20057_01697.jpeg - Authentic: 0.9999939203262329
Au_pla_30118.jpg - Authentic: 1.0
Au_sec_30397.jpg - Authentic: 0.9999980330467224
Au_txt_00003.jpg - Authentic: 1.0
Au_pla_30656.jpg - Authentic: 1.0
Tp_D_CND_M_N

Tp_S_NRN_S_B_ani00037_ani00037_00157.jpeg - Authentic: 1.0
Tp_S_NRD_S_N_art20081_art20081_01887.jpeg - Manipulated: 0.004690966103225946
Au_pla_30654.jpg - Authentic: 1.0
Au_txt_00029.jpg - Authentic: 0.9999991059303284
Au_pla_30640.jpg - Authentic: 1.0
Au_arc_30803.jpg - Authentic: 1.0
Au_nat_00063.jpg - Authentic: 0.9999999403953552
Au_pla_30126.jpg - Authentic: 1.0
Au_nat_30188.jpg - Authentic: 1.0
Au_sec_30381.jpg - Authentic: 0.9999971985816956
Tp_S_NRN_S_N_arc20065_arc20065_01705.jpeg - Manipulated: 0.17838534712791443
Tp_S_NRN_S_N_cha20042_cha20042_02041.jpeg - Manipulated: 0.1365605741739273
Au_pla_30668.jpg - Authentic: 1.0
Au_txt_00015.jpg - Authentic: 1.0
Au_txt_00001.jpg - Authentic: 0.9999988079071045
Au_sec_30395.jpg - Authentic: 0.9999988675117493
Au_nat_30177.jpg - Authentic: 1.0
Tp_D_CRD_M_N_arc10116_arc10116_10762.jpg - Manipulated: 0.04509485885500908
Au_pla_00032.jpg - Authentic: 0.9999997019767761
Au_sec_30418.jpg - Authentic: 0.9999996423721313
Au_pla_30697.jpg - 

Au_nat_30002.jpg - Authentic: 1.0
Tp_D_CNN_M_N_ani00023_ani00024_10205.jpeg - Authentic: 0.9999998807907104
Au_pla_30709.jpg - Authentic: 1.0
Au_sec_30586.jpg - Authentic: 1.0
Au_arc_30792.jpg - Authentic: 0.9999719858169556
Tp_D_CRN_M_N_art00097_art00049_11758.jpg - Authentic: 0.9995521903038025
Tp_S_NRD_S_N_ani20011_ani20011_02054.jpeg - Authentic: 0.9754850268363953
Au_arc_30786.jpg - Authentic: 0.9999999403953552
Au_sec_30592.jpg - Authentic: 1.0
Tp_S_NRN_S_N_cha00039_cha00039_11013.jpg - Authentic: 0.9989097118377686
Au_pla_30735.jpg - Authentic: 0.9999542236328125
Au_pla_30053.jpg - Authentic: 1.0
Tp_S_NRN_S_N_arc20070_arc20070_01710.jpeg - Manipulated: 0.09110350161790848
Tp_D_CRN_S_O_cha00030_ani00068_10042.jpeg - Authentic: 1.0
Au_pla_30047.jpg - Authentic: 1.0
Au_pla_30721.jpg - Authentic: 1.0
Au_nat_30228.jpg - Authentic: 1.0
Au_sec_30021.jpg - Authentic: 1.0
Au_arc_30553.jpg - Authentic: 1.0
Au_arc_30547.jpg - Authentic: 1.0
Tp_S_NRN_S_N_txt00071_txt00071_11300.jpg - Manipu

Au_sec_30145.jpg - Authentic: 1.0
Au_sec_00086.jpg - Authentic: 0.9999985098838806
Tp_S_NRN_S_B_art00084_art00084_00929.jpeg - Authentic: 0.9999994039535522
Au_sec_30623.jpg - Authentic: 1.0
Tp_S_NRD_S_N_ind00081_ind00081_10670.jpg - Authentic: 0.9731937050819397
Tp_S_NRN_S_N_cha00037_cha00037_00357.jpeg - Authentic: 1.0
Au_sec_30637.jpg - Authentic: 1.0
Au_sec_30151.jpg - Authentic: 1.0
Au_sec_00092.jpg - Authentic: 0.9999999403953552
Au_nat_30358.jpg - Authentic: 1.0
Tp_D_NND_L_B_arc00033_nat00095_00253.jpeg - Authentic: 0.9999285936355591
Au_sec_30179.jpg - Authentic: 1.0
Tp_D_CRN_S_N_sec00052_sec00055_11238.jpg - Authentic: 0.6184201240539551
Au_nat_30370.jpg - Authentic: 1.0
Au_nat_30416.jpg - Authentic: 1.0
Au_pla_30490.jpg - Authentic: 1.0
Au_pla_30484.jpg - Authentic: 1.0
Au_nat_30402.jpg - Authentic: 1.0
Au_nat_30364.jpg - Authentic: 1.0
Au_nat_30403.jpg - Authentic: 1.0
Au_pla_30485.jpg - Authentic: 1.0
Tp_S_NRN_M_N_art00042_art00042_10543.jpeg - Authentic: 0.9977967143058777

...

Au_pla_30661.jpg - Authentic: 1.0
Tp_D_CRN_S_N_art00067_art00092_11808.jpg - Manipulated: 0.14028465747833252
Tp_S_NRN_M_N_cha10208_cha10208_12338.jpg - Manipulated: 0.00011402132804505527
Au_sec_30388.jpg - Authentic: 1.0
Au_pla_30107.jpg - Authentic: 1.0
Au_nat_30181.jpg - Authentic: 1.0
Au_pla_30649.jpg - Authentic: 1.0
Au_txt_00034.jpg - Authentic: 1.0
Tp_S_NRN_S_N_sec20094_sec20094_01684.jpeg - Authentic: 0.9999452233314514
Au_txt_00020.jpg - Authentic: 0.9999998211860657
Au_pla_00013.jpg - Authentic: 1.0
Au_nat_30156.jpg - Authentic: 1.0
Au_nat_00095.jpg - Authentic: 0.9499393701553345
Au_nat_30630.jpg - Authentic: 0.9999969601631165
Au_sec_30439.jpg - Authentic: 1.0
Au_nat_30624.jpg - Authentic: 1.0
Au_arc_30639.jpg - Authentic: 0.9999998807907104
Tp_S_NRN_S_B_cha20001_cha20001_01504.jpeg - Manipulated: 0.23838604986667633
Au_nat_30142.jpg - Authentic: 0.9999976754188538
Au_nat_00081.jpg - Authentic: 0.9999985098838806
Au_pla_00007.jpg - Authentic: 1.0
Tp_S_NRN_S_N_pla10126_pla1

Au_pla_30066.jpg - Authentic: 1.0
Au_pla_30700.jpg - Authentic: 0.9999995827674866
Tp_D_NNN_M_B_art00023_nat00095_11877.jpg - Manipulated: 0.031822606921195984
Au_nat_30209.jpg - Authentic: 1.0
Au_arc_30572.jpg - Authentic: 1.0
Au_arc_30566.jpg - Authentic: 1.0
Tp_D_NNN_M_B_nat00086_nat00097_11284.jpg - Manipulated: 0.013252164237201214
Au_sec_30014.jpg - Authentic: 0.9999998807907104
Tp_D_NND_S_N_cha00010_arc00086_00330.jpeg - Authentic: 0.9999999403953552
Au_pla_20088.jpg - Manipulated: 0.09418146312236786
Au_nat_20026.jpg - Manipulated: 0.011836795136332512
Au_nat_30235.jpg - Authentic: 1.0
Au_cha_30020.jpg - Authentic: 1.0
Au_nat_30553.jpg - Authentic: 1.0
Au_nat_30547.jpg - Authentic: 1.0
Au_cha_30034.jpg - Authentic: 1.0
Au_sec_30028.jpg - Authentic: 0.9999999403953552
Tp_S_NRN_S_O_cha00035_cha00067_11734.jpg - Authentic: 0.9894145131111145
Au_nat_20032.jpg - Manipulated: 0.1505400836467743
Au_nat_30221.jpg - Authentic: 0.9999988079071045
Au_sec_00100.jpg - Authentic: 1.0
Au_pla_

Au_nat_30351.jpg - Authentic: 1.0
Au_sec_30158.jpg - Authentic: 1.0
Au_cha_00087.jpg - Authentic: 0.9966074824333191
Au_nat_30437.jpg - Authentic: 0.9999980926513672
Au_nat_30423.jpg - Authentic: 1.0
Au_nat_30345.jpg - Authentic: 1.0
Au_nat_10138.jpeg - Manipulated: 0.31339821219444275
Au_nat_30422.jpg - Authentic: 1.0
Au_nat_30344.jpg - Authentic: 1.0
Tp_S_NRN_S_N_cha00060_cha00060_11023.jpg - Manipulated: 0.004806690849363804
Tp_D_CRN_S_N_nat10145_ani00022_11986.jpg - Manipulated: 0.0017428932478651404
Au_sec_30159.jpg - Authentic: 1.0
Au_nat_30350.jpg - Authentic: 1.0
Au_nat_30436.jpg - Authentic: 1.0
Tp_S_NRN_S_O_arc00055_arc00055_01071.jpeg - Authentic: 0.9996348023414612
Au_cha_00086.jpg - Authentic: 0.9999998807907104
Au_sec_30617.jpg - Authentic: 1.0
Au_pla_30498.jpg - Authentic: 1.0
Au_sec_30171.jpg - Authentic: 1.0
Au_nat_30378.jpg - Authentic: 1.0
Tp_S_NRN_S_N_art00077_art00077_11712.jpg - Manipulated: 0.006878319196403027
Au_sec_30165.jpg - Authentic: 1.0
Tp_D_CRN_S_N_art10

Au_nat_20035.jpg - Manipulated: 0.0138853145763278
Au_nat_30226.jpg - Authentic: 1.0
Au_pla_30049.jpg - Authentic: 1.0
Tp_S_NRN_S_N_sec20023_sec20023_01613.jpeg - Authentic: 0.9806616902351379
Au_sec_10102.jpg - Manipulated: 0.06440776586532593
Tp_D_CRN_S_N_nat10159_ani00097_12063.jpg - Manipulated: 0.008801789954304695
Au_arc_30788.jpg - Authentic: 1.0
Au_pla_30713.jpg - Authentic: 1.0
Au_pla_30075.jpg - Authentic: 1.0
Tp_S_NRD_S_B_art00024_art00024_00923.jpeg - Authentic: 0.9999805688858032
Au_pla_30061.jpg - Authentic: 1.0
Au_sec_30588.jpg - Authentic: 1.0
Au_pla_30707.jpg - Authentic: 1.0
Au_sec_30563.jpg - Authentic: 0.9999992847442627
Au_sec_30205.jpg - Authentic: 1.0
Au_sec_20016.jpg - Authentic: 0.8733115196228027
Au_nat_30018.jpg - Authentic: 0.9999812245368958
Tp_S_NRN_S_N_arc20007_arc20007_02200.jpeg - Authentic: 0.770016074180603
Au_sec_30211.jpg - Authentic: 1.0
Au_sec_20002.jpg - Manipulated: 0.09042952954769135
Au_sec_30577.jpg - Authentic: 1.0
Tp_S_NRN_S_N_arc20001_arc2

Au_nat_00079.jpg - Authentic: 0.9999998807907104
Au_nat_30025.jpg - Authentic: 1.0
Au_cha_30556.jpg - Authentic: 1.0
Tp_S_NRN_S_N_ind20030_ind20030_01771.jpeg - Authentic: 0.9838879704475403
Au_sec_30238.jpg - Authentic: 1.0
Au_nat_30031.jpg - Authentic: 0.9999950528144836
Au_sec_20003.jpg - Manipulated: 0.11840737611055374
Au_sec_30210.jpg - Authentic: 1.0
Tp_S_NRN_S_N_nat10147_nat10147_11995.jpg - Manipulated: 0.005478383507579565
Au_nat_30019.jpg - Authentic: 0.9999968409538269
Au_sec_30576.jpg - Authentic: 1.0
Tp_S_NRN_S_N_nat00082_nat00082_00986.jpeg - Authentic: 0.9998846054077148
Au_sec_30562.jpg - Authentic: 1.0
Tp_S_NRN_S_B_cha00086_art00012_00406.jpeg - Authentic: 0.9999999403953552
Au_sec_20017.jpg - Manipulated: 0.22341355681419373
Au_sec_30204.jpg - Authentic: 1.0
Au_pla_30060.jpg - Authentic: 1.0
Au_pla_30706.jpg - Authentic: 1.0
Au_sec_30589.jpg - Authentic: 1.0
Au_pla_30712.jpg - Authentic: 0.9999998807907104
Au_arc_30789.jpg - Authentic: 1.0
Au_pla_30074.jpg - Authenti